<a href="https://colab.research.google.com/github/mariasald/avd-2024-01/blob/main/Atividade_Pr%C3%A1tica_4_O_Problema_do_Transiente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Alunos**

* Gabriel César Tavares Ferreira - 21854868
* Maria Luiza Saldanha - 22153140

#**Bibliotecas Utilizadas**

In [1]:
import math
import numpy as np
import random
import matplotlib.pyplot as plt
import scipy.stats
from scipy.special import factorial
from tabulate import tabulate

#**BibAD (1o Trabaho)**

In [2]:
def mediaAritmetica(nums):
    return round((sum(nums) / len(nums)), 3)

def var_amostral(nums):
  med = mediaAritmetica(nums)
  quad = np.sum(np.fromiter(((x - med) * (x - med) for x in nums), dtype=float))
  var = quad / len(nums)
  return round(var)

def desvio_padrao(nums):
  return round(math.sqrt(var_amostral(nums)))

def intervaloConfianca(nums, z):
    nivelConfianca = z / 100
    a = 1 - nivelConfianca
    z_a = 1 - (a / 2)
    distribuicao = scipy.stats.norm.ppf(z_a)
    erroPadrao = distribuicao * desvio_padrao(nums) / math.sqrt(len(nums))

    return [round((mediaAritmetica(nums) - erroPadrao), 3), round((mediaAritmetica(nums) + erroPadrao), 3)]

#**BibAD (3° Trabalho)**

In [3]:
def va_exponencial(media):
  u = random.random()
  x = -media * math.log (1 - u)
  return round(x,3)

def e_x(taxa_chegada, taxa_servico):
  rho = taxa_chegada / taxa_servico
  E_X = (rho / taxa_servico) / (1 - rho)
  return E_X

print(va_exponencial(1/9)) #nao esquecer de ver quando receber taxa e quando receber media!!!!!

def fila_mm1(taxa_chegada, taxa_servico, clientes_total):
    E_X = e_x(taxa_chegada, taxa_servico)

    tempos_chegada = np.cumsum([va_exponencial(1/taxa_chegada) for _ in range(clientes_total)])
    tempos_servico = [va_exponencial(1/taxa_servico) for _ in range(clientes_total)]

    tempos_espera = np.zeros(clientes_total)
    tempo_livre = 0

    for i in range(clientes_total):
        if tempos_chegada[i] > tempo_livre:
            tempos_espera[i] = 0
            tempo_livre = tempos_chegada[i] + tempos_servico[i]
        else:
            tempos_espera[i] = tempo_livre - tempos_chegada[i]
            tempo_livre += tempos_servico[i]

    X_n = np.mean(tempos_espera)
    confidencia = 0.95
    z = scipy.stats.norm.ppf(1 - (1 - confidencia) / 2)
    erro_padrao = np.std(tempos_espera, ddof=1) / np.sqrt(clientes_total)
    intervalo_confianca = (X_n - z * erro_padrao, X_n + z * erro_padrao)

    return round(X_n, 3)



0.277


#**Parte 1**

$\bar{X}(n)=\frac{\sum_{i=1}^n x_i}{n}$

* Estimar $\bar{X}(n)$, o tempo médio de espera em uma fila M/M/1 ($n=10^3,λ=9,μ=10$).
* Calcular o valor teórico: $E[X]=\rho \frac{1/\mu}{1-\rho}, \; onde \; \rho = \frac{\lambda}{\mu}$
* Computar o viés (bias): $B=\bar{X}(n)−E[X]$.
* Repetir a simulação r vezes (r=30).
* Anotar os resultados em uma tabela.
* Analisar os resultados.

In [4]:
resultados = fila_mm1(9, 10, 10**3)
print(resultados)

0.678


In [5]:
resultados1 = e_x(9, 10)
print(resultados1)

0.9000000000000001


In [6]:
bias = resultados-resultados1
print(bias)

-0.2220000000000001


In [7]:
result_t = [["X", "B"]]
for i in range(30):
  aux = []
  result = fila_mm1(9, 10, 10**3)
  bias = result - e_x(9, 10)
  aux.append(result)
  aux.append(bias)
  result_t.append(aux)

tabela = tabulate(result_t, headers="firstrow", tablefmt="grid")
print(tabela)

+-------+--------+
|     X |      B |
+=======+========+
| 1.068 |  0.168 |
+-------+--------+
| 1.047 |  0.147 |
+-------+--------+
| 0.868 | -0.032 |
+-------+--------+
| 1.167 |  0.267 |
+-------+--------+
| 0.506 | -0.394 |
+-------+--------+
| 0.76  | -0.14  |
+-------+--------+
| 0.71  | -0.19  |
+-------+--------+
| 1.16  |  0.26  |
+-------+--------+
| 0.981 |  0.081 |
+-------+--------+
| 2.311 |  1.411 |
+-------+--------+
| 1.353 |  0.453 |
+-------+--------+
| 0.839 | -0.061 |
+-------+--------+
| 0.571 | -0.329 |
+-------+--------+
| 0.54  | -0.36  |
+-------+--------+
| 0.444 | -0.456 |
+-------+--------+
| 0.64  | -0.26  |
+-------+--------+
| 2.761 |  1.861 |
+-------+--------+
| 0.615 | -0.285 |
+-------+--------+
| 0.96  |  0.06  |
+-------+--------+
| 0.311 | -0.589 |
+-------+--------+
| 0.687 | -0.213 |
+-------+--------+
| 0.797 | -0.103 |
+-------+--------+
| 0.882 | -0.018 |
+-------+--------+
| 0.937 |  0.037 |
+-------+--------+
| 1.003 |  0.103 |
+-------+---

Análise

#**Parte 2**

* Simular o tempo médio de espera em uma fila M/M/1 (λ=9,μ=10).
* Eliminar o transiente usando as heurísticas de (a) Conway e (b) Fishman.
* Após eliminar o transiente, coletar n valores no estado estacionário ( n=103).
* Calcular $\bar{X}(n)$, o tempo médio de espera na fila e o viés (B).
* Repetir a simulação r vezes (r=30).
* Mostrar os resultados em uma tabela.
* Analisar os resultados.

##Heurística de Conway
Conway sugere descartar os primeiros 20% dos dados para eliminar o transiente.

##Heurística de Fishman
Fishman sugere descartar dados até que o sistema atinja um estado estacionário.


In [13]:
resultados = fila_mm1(9, 10, 10**3)
print(resultados)

0.668


In [14]:
resultadosTeorico = e_x(9, 10)
print(resultadosTeorico)

0.9000000000000001


In [15]:
bias = resultados-resultadosTeorico
print(bias)

-0.2320000000000001


In [16]:
def heuristica_conway(dados):
    return dados[int(0.2 * len(dados)):]

def heuristica_fishman(dados):
    tamanho_janela = 50
    for i in range(len(dados) - tamanho_janela):
        janela = dados[i:i+tamanho_janela]
        media_janela = np.mean(janela)
        if np.all(np.abs(janela - media_janela) < 0.01 * media_janela):
            return dados[i+tamanho_janela:]
    return dados

def fila_mm1_com_heuristicas(taxa_chegada, taxa_servico, clientes_total):
    E_X = e_x(taxa_chegada, taxa_servico)
    tempos_chegada = np.cumsum([va_exponencial(1/taxa_chegada) for _ in range(clientes_total)])
    tempos_servico = [va_exponencial(1/taxa_servico) for _ in range(clientes_total)]

    tempos_espera = np.zeros(clientes_total)
    tempo_livre = 0

    for i in range(clientes_total):
        if tempos_chegada[i] > tempo_livre:
            tempos_espera[i] = 0
            tempo_livre = tempos_chegada[i] + tempos_servico[i]
        else:
            tempos_espera[i] = tempo_livre - tempos_chegada[i]
            tempo_livre += tempos_servico[i]

    tempos_espera_conway = heuristica_conway(tempos_espera)
    tempos_espera_fishman = heuristica_fishman(tempos_espera)

    X_n_conway = np.mean(tempos_espera_conway)
    X_n_fishman = np.mean(tempos_espera_fishman)

    return round(X_n_conway, 3), round(X_n_fishman, 3)


In [17]:
taxa_chegada = 9
taxa_servico = 10
clientes_total = 10**3

E_X_teorico = e_x(taxa_chegada, taxa_servico)

resultados_conway = [["X_Conway", "B_Conway"]]
resultados_fishman = [["X_Fishman", "B_Fishman"]]
for i in range(30):
    resultado_conway, resultado_fishman = fila_mm1_com_heuristicas(taxa_chegada, taxa_servico, clientes_total)
    vies_conway = resultado_conway - E_X_teorico
    vies_fishman = resultado_fishman - E_X_teorico
    resultados_conway.append([resultado_conway, vies_conway])
    resultados_fishman.append([resultado_fishman, vies_fishman])

tabela_conway = tabulate(resultados_conway, headers="firstrow", tablefmt="grid")
tabela_fishman = tabulate(resultados_fishman, headers="firstrow", tablefmt="grid")

print("Tabela de Resultados - Conway")
print(tabela_conway)
print("\nTabela de Resultados - Fishman")
print(tabela_fishman)

Tabela de Resultados - Conway
+------------+------------+
|   X_Conway |   B_Conway |
+============+============+
|      0.808 |     -0.092 |
+------------+------------+
|      0.515 |     -0.385 |
+------------+------------+
|      0.659 |     -0.241 |
+------------+------------+
|      0.803 |     -0.097 |
+------------+------------+
|      1.174 |      0.274 |
+------------+------------+
|      0.564 |     -0.336 |
+------------+------------+
|      1.905 |      1.005 |
+------------+------------+
|      1.601 |      0.701 |
+------------+------------+
|      0.616 |     -0.284 |
+------------+------------+
|      0.464 |     -0.436 |
+------------+------------+
|      0.451 |     -0.449 |
+------------+------------+
|      0.807 |     -0.093 |
+------------+------------+
|      0.639 |     -0.261 |
+------------+------------+
|      1.508 |      0.608 |
+------------+------------+
|      3.63  |      2.73  |
+------------+------------+
|      0.683 |     -0.217 |
+------------+----

#**Parte 3**

* Executar uma simulação de horizonte infinito da fila M/M/1.
* Eliminar o transiente usando a heurística MSER-5Y.
* Estimar o tempo médio de espera na fila e o respectivo intervalo de confiança de 95%.
* A regra de parada é a precisão relativa de 5%.
* Cenário    I:  λ=7 clientes por segundo; μ=10 clientes por segundo.
* Cenário   II:  λ=8 clientes por segundo; μ=10 clientes por segundo.
* Cenário  III:  λ=9 clientes por segundo; μ=10 clientes por segundo.
* Cenário IV:  λ=9,5 clientes por segundo; μ=10 clientes por segundo.
* Plotar um gráfico comparativo dos quatro cenários.
* Plotar uma linha horizontal para o valor teórico esperado em cada cenário.
* Analisar os resultados.